In [14]:
import yfinance as yf

# Hourly — will be empty because Yahoo doesn't retain 1h that far back
df_hour = yf.download("TSLA", start="2024-08-12", end="2024-08-15", interval="1h")


C:\Users\sriha\AppData\Local\Temp\ipykernel_53060\4228013507.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_hour = yf.download("TSLA", start="2024-08-12", end="2024-08-15", interval="1h")
[*********************100%***********************]  1 of 1 completed


In [27]:
import io
import requests
import pandas as pd
import matplotlib.pyplot as plt

URL = "https://www.alphavantage.co/query"
API_KEY = "YOUR_KEY"   # <-- put your Alpha Vantage key here

def get_intraday_plot(symbol: str, start_date: str, end_date: str, interval: str = "30min"):
    """
    Fetch intraday OHLCV data for a stock between start_date and end_date
    and plot closing prices.

    Args:
        symbol (str): Stock ticker, e.g. "TSLA"
        start_date (str): Start date, format YYYY-MM-DD
        end_date (str): End date, format YYYY-MM-DD
        interval (str): Interval string, e.g. "30min", "60min", "15min"
    """
    # use historical slice by month
    month = pd.to_datetime(start_date).strftime("%Y-%m")

    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": interval,
        "month": month,
        "outputsize": "full",
        "apikey": API_KEY,
        "datatype": "csv",
    }

    r = requests.get(URL, params=params, timeout=60)
    txt = r.text
    if txt.lstrip().startswith("{"):
        raise RuntimeError(f"API returned error: {txt[:200]} ...")

    df = pd.read_csv(io.StringIO(txt), parse_dates=["timestamp"]).sort_values("timestamp")

    # filter to the window
    mask = (df["timestamp"].dt.date >= pd.to_datetime(start_date).date()) & \
           (df["timestamp"].dt.date <= pd.to_datetime(end_date).date())
    df = df.loc[mask].reset_index(drop=True)

    if df.empty:
        print("No data found for this range")
        return df

    # plot closing prices
    plt.figure(figsize=(12,6))
    plt.plot(df["timestamp"], df["close"], marker="o", linestyle="-")
    plt.title(f"{symbol} {interval} Close Prices ({start_date} → {end_date})")
    plt.xlabel("Time")
    plt.ylabel("Close Price (USD)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

    return df

df_plug = get_intraday_plot("PLUG", "2021-03-16", "2021-03-17", interval="30min")
df_plug
# PG&E Corporation (PCG) — 2019-01-14, 30-min bars
# PG&E Corporation (PCG) — 2019-01-14, 30-min bars
df_sivb

RuntimeError: API returned error: {
    "Information": "We have detected your API key as YOUR_KEY and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/prem ...

In [28]:
# pip install edgartools
from edgar import Company, set_identity

set_identity("Sri <sri.jerripotula@gmail.com>")  # SEC-friendly identity

def get_8k_links(ticker: str, start_date: str, end_date: str, include_amendments: bool = False):
    """
    Return [(filing_date, filing_url)] for 8-Ks in the date range.
    Dates are 'YYYY-MM-DD'.
    """
    co = Company(ticker)
    filings = co.get_filings(form="8-K", amendments=include_amendments)
    filings = filings.filter(filing_date=f"{start_date}:{end_date}")
    # sort oldest -> newest
    filings = sorted(list(filings), key=lambda f: f.filing_date)
    return [(f.filing_date, f.filing_url) for f in filings]

# ---- Your GE example (2018-10-01 to 2018-10-04) ----
links = get_8k_links("GE", "2018-10-01", "2018-10-04")
for d, url in links:
    print(f"{d} — {url}")
from IPython.display import HTML, display

def show_8k_links(ticker, start_date, end_date, include_amendments=False):
    data = get_8k_links(ticker, start_date, end_date, include_amendments)
    if not data:
        display(HTML("<p>No 8-Ks found.</p>"))
        return
    html = "<ul>" + "".join(
        f'<li>{d} — <a href="{u}" target="_blank">{u}</a></li>' for d, u in data
    ) + "</ul>"
    display(HTML(html))

# Example:
show_8k_links("PLUG", "2021-03-16", "2021-03-17")


2018-10-04 — https://www.sec.gov/Archives/edgar/data/40545/000004054518000074/ge8-k100418.htm
